In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]: 


from ast import arg
import time
import argparse
import numpy as np
import torch
from models.GCN import GCN
from models.GCN_Encoder import GCN_Encoder
from torch_geometric.datasets import Planetoid, WebKB, WikipediaNetwork,Reddit,Reddit2,Flickr,Yelp,PPI
from torch_geometric.utils import to_dense_adj,dense_to_sparse

# from ogb.nodeproppred import PygNodePropPredDataset
# from torch_geometric.loader import DataLoader
from help_funcs import prune_unrelated_edge,prune_unrelated_edge_isolated,select_target_nodes
import help_funcs
import scipy.sparse as sp

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--debug', action='store_true',
        default=True, help='debug mode')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--seed', type=int, default=10, help='Random seed.')
parser.add_argument('--model', type=str, default='GCN', help='model',
                    choices=['GCN','GAT','GraphSage','GIN'])
parser.add_argument('--dataset', type=str, default='Cora', 
                    help='Dataset',
                    choices=['Cora','Citeseer','Pubmed','PPI','Flickr','ogbn-arxiv','Reddit','Reddit2','Yelp'])
parser.add_argument('--train_lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=32,
                    help='Number of hidden units.')
parser.add_argument('--thrd', type=float, default=0.5)
parser.add_argument('--target_class', type=int, default=0)
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('--epochs', type=int,  default=200, help='Number of epochs to train benign and backdoor model.')
parser.add_argument('--trojan_epochs', type=int,  default=200, help='Number of epochs to train trigger generator.')
parser.add_argument('--inner', type=int,  default=1, help='Number of inner')
# backdoor setting
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--trigger_size', type=int, default=3,
                    help='tirgger_size')
parser.add_argument('--vs_ratio', type=float, default=0.005,
                    help="ratio of poisoning nodes relative to the full graph")
# defense setting
parser.add_argument('--defense_mode', type=str, default="guard",
                    choices=['prune', 'isolate', 'none', 'guard', 'median'],
                    help="Mode of defense")
parser.add_argument('--prune_thr', type=float, default=0.15,
                    help="Threshold of prunning edges")
parser.add_argument('--target_loss_weight', type=float, default=1,
                    help="Weight of optimize outter trigger generator")
parser.add_argument('--homo_loss_weight', type=float, default=3,
                    help="Weight of optimize similarity loss")
parser.add_argument('--homo_boost_thrd', type=float, default=0.5,
                    help="Threshold of increase similarity")
# attack setting
parser.add_argument('--dis_weight', type=float, default=1,
                    help="Weight of cluster distance")
parser.add_argument('--attack_method', type=str, default='Basic',
                    choices=['Rand_Gene','Rand_Samp','Basic','None','TDGIA','AGIA'],
                    help='Method to select idx_attach for training trojan model (none means randomly select)')
parser.add_argument('--trigger_prob', type=float, default=0.5,
                    help="The probability to generate the trigger's edges in random method")
parser.add_argument('--selection_method', type=str, default='none',
                    choices=['loss','conf','cluster','none','cluster_degree'],
                    help='Method to select idx_attach for training trojan model (none means randomly select)')
parser.add_argument('--test_model', type=str, default='GCN',
                    choices=['GCN','GAT','GraphSage','GIN'],
                    help='Model used to attack')
parser.add_argument('--evaluate_mode', type=str, default='overall',
                    choices=['overall','1by1'],
                    help='Model used to attack')
# GPU setting
parser.add_argument('--device_id', type=int, default=0,
                    help="Threshold of prunning edges")
# args = parser.parse_args()
args = parser.parse_known_args()[0]
args.cuda =  not args.no_cuda and torch.cuda.is_available()
device = torch.device(('cuda:{}' if torch.cuda.is_available() else 'cpu').format(args.device_id))
# device2 = torch.device(('cuda:{}' if torch.cuda.is_available() else 'cpu').format(args.device_id+1))

# np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
print(args)
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
    #  random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(args.seed)
#%%
from torch_geometric.utils import to_undirected
import torch_geometric.transforms as T
transform = T.Compose([T.NormalizeFeatures()])

np.random.seed(11) # fix the random seed is important
if(args.dataset == 'Cora' or args.dataset == 'Citeseer' or args.dataset == 'Pubmed'):
    dataset = Planetoid(root='./data/', \
                        name=args.dataset,\
                        transform=transform)
elif(args.dataset == 'Flickr'):
    dataset = Flickr(root='./data/Flickr/', \
                    transform=transform)
elif(args.dataset == 'PPI'):
    dataset = PPI(root='./data/PPI/', 
                split='train', transform=None)
elif(args.dataset == 'Reddit2'):
    dataset = Reddit2(root='./data/Reddit2/', \
                    transform=transform)
elif(args.dataset == 'ogbn-arxiv'):
    # Download and process data at './dataset/ogbg_molhiv/'
    dataset = PygNodePropPredDataset(name = 'ogbn-arxiv', root='./data/')
    split_idx = dataset.get_idx_split() 
elif(args.dataset == 'Yelp'):
    # Download and process data at './dataset/ogbg_molhiv/'
    dataset = Yelp(root='./data/Yelp/')
    # idx_train, idx_val, idx_test = split_idx["train"], split_idx["valid"], split_idx["test"]


data = dataset[0].to(device)

if(args.dataset == 'ogbn-arxiv'):
    nNode = data.x.shape[0]
    setattr(data,'train_mask',torch.zeros(nNode, dtype=torch.bool).to(device))
    # dataset[0].train_mask = torch.zeros(nEdge, dtype=torch.bool).to(device)
    data.val_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
    data.test_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
    data.y = data.y.squeeze(1)
# we build our own train test split 
from utils import get_split
data, idx_train, idx_val, idx_clean_test, idx_atk = get_split(args,data,device)

print(data)

#%%
from torch_geometric.utils import to_undirected
from utils import subgraph
data.edge_index = to_undirected(data.edge_index)
train_edge_index,_, edge_mask = subgraph(torch.bitwise_not(data.test_mask),data.edge_index,relabel_nodes=False)
mask_edge_index = data.edge_index[:,torch.bitwise_not(edge_mask)]

# In[3]:

# In[6]: 
import os
from models.backdoor import model_construct
benign_modelpath = './modelpath/{}_{}_benign.pth'.format(args.model, args.dataset)
if(os.path.exists(benign_modelpath)):
    # load existing benign model
    benign_model = torch.load(benign_modelpath)
    benign_model = benign_model.to(device)
    # edge_weights = torch.ones([data.edge_index.shape[1]],device=device,dtype=torch.float)
    print("Loading benign {} model Finished!".format(args.model))
else:
    benign_model = model_construct(args,args.test_model,data,device).to(device) 
    t_total = time.time()
    print("Length of training set: {}".format(len(idx_train)))
    benign_model.fit(data.x, train_edge_index, None, data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
    print("Training benign model Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    # Save trained model
    # torch.save(benign_model, benign_modelpath)
    # print("Benign model saved at {}".format(benign_modelpath))

# In[7]:

benign_ca = benign_model.test(data.x, data.edge_index, None, data.y,idx_clean_test)
print("Benign CA: {:.4f}".format(benign_ca))
benign_model = benign_model.cpu()

# In[9]:

# from sklearn_extra import cluster
# from models.backdoor import obtain_attach_nodes,Backdoor, obtain_attach_nodes_by_cluster_degree, obtain_attach_nodes_by_cluster_gpu,obtain_attach_nodes_by_influential,obtain_attach_nodes_by_cluster,cluster_distance_selection,cluster_degree_selection

# from kmeans_pytorch import kmeans, kmeans_predict
# from models.backdoor import defend_baseline_construct

from sklearn_extra import cluster
from models.backdoor import Backdoor, defend_baseline_construct
from heuristic_selection import obtain_attach_nodes, obtain_attach_nodes_by_cluster_degree, obtain_attach_nodes_by_cluster_gpu,obtain_attach_nodes_by_influential,obtain_attach_nodes_by_cluster,cluster_distance_selection,cluster_degree_selection

from kmeans_pytorch import kmeans, kmeans_predict

# filter out the unlabeled nodes except from training nodes and testing nodes, nonzero() is to get index, flatten is to get 1-d tensor
unlabeled_idx = (torch.bitwise_not(data.test_mask)&torch.bitwise_not(data.train_mask)).nonzero().flatten()
size = int((len(data.test_mask)-data.test_mask.sum())*args.vs_ratio)
# here is randomly select poison nodes from unlabeled nodes
if(args.selection_method == 'none'):
    idx_attach = obtain_attach_nodes(args,unlabeled_idx,size)
elif(args.selection_method == 'loss' or args.selection_method == 'conf'):
    idx_attach = obtain_attach_nodes_by_influential(args,benign_model,unlabeled_idx.cpu().tolist(),data.x,train_edge_index,None,data.y,device,size,selected_way=args.selection_method)
    idx_attach = torch.LongTensor(idx_attach).to(device)
elif(args.selection_method == 'cluster'):
    idx_attach = cluster_distance_selection(args,data,idx_train,idx_val,idx_clean_test,unlabeled_idx,train_edge_index,size,device)
    idx_attach = torch.LongTensor(idx_attach).to(device)
elif(args.selection_method == 'cluster_degree'):
    idx_attach = cluster_degree_selection(args,data,idx_train,idx_val,idx_clean_test,unlabeled_idx,train_edge_index,size,device)
    idx_attach = torch.LongTensor(idx_attach).to(device)

# In[10]:
# train trigger generator 
model = Backdoor(args,device)
if(args.attack_method == 'Basic'):
    model.fit(data.x, train_edge_index, None, data.y, idx_train,idx_attach, unlabeled_idx)
    poison_x, poison_edge_index, poison_edge_weights, poison_labels = model.get_poisoned()
elif(args.attack_method == 'Rand_Gene' or args.attack_method == 'Rand_Samp'):
    model.fit_rand(data.x, train_edge_index, None, data.y, idx_train,idx_attach, unlabeled_idx)
    poison_x, poison_edge_index, poison_edge_weights, poison_labels = model.get_poisoned_rand()
elif(args.attack_method == 'None'):
    train_edge_weights = torch.ones([train_edge_index.shape[1]],device=device,dtype=torch.float)
    poison_x, poison_edge_index, poison_edge_weights, poison_labels = data.x.clone(), train_edge_index.clone(), train_edge_weights, data.y.clone()
elif(args.attack_method == 'TDGIA' or args.attack_method == 'AGIA'):
    train_edge_weights = torch.ones([train_edge_index.shape[1]],device=device,dtype=torch.float)
    poison_x, poison_edge_index, poison_edge_weights, poison_labels = data.x.clone(), train_edge_index.clone(), train_edge_weights, data.y.clone()
    # build surrogate model
    surrogate_model = model_construct(args,args.model,data,device).to(device) 
    t_total = time.time()
    # print("Length of training set: {}".format(len(idx_train)))
    print("Training Surrogate Model...")
    surrogate_model.fit(data.x, train_edge_index, None, data.y, idx_train, idx_val,train_iters=args.epochs,verbose=True)
    print("Training Completed...")
# In[12]:
if(args.defense_mode == 'prune'):
    poison_edge_index,poison_edge_weights = prune_unrelated_edge(args,poison_edge_index,poison_edge_weights,poison_x,device)
    bkd_tn_nodes = torch.cat([idx_train,idx_attach]).to(device)
elif(args.defense_mode == 'isolate'):
    poison_edge_index,poison_edge_weights,rel_nodes = prune_unrelated_edge_isolated(args,poison_edge_index,poison_edge_weights,poison_x,device)
    bkd_tn_nodes = torch.cat([idx_train,idx_attach]).tolist()
    bkd_tn_nodes = torch.LongTensor(list(set(bkd_tn_nodes) - set(rel_nodes))).to(device)
else:
    bkd_tn_nodes = torch.cat([idx_train,idx_attach]).to(device)

print("precent of left attach nodes: {:.3f}"\
    .format(len(set(bkd_tn_nodes.tolist()) & set(idx_attach.tolist()))/len(idx_attach)))
#%%
# test_model = model_construct(args,args.test_model,data,device).to(device) 
test_model = defend_baseline_construct(args,args.defense_mode,args.test_model,data,device).to(device) 
test_model.fit(poison_x, poison_edge_index, poison_edge_weights, poison_labels, bkd_tn_nodes, idx_val,train_iters=args.epochs,verbose=False)

output = test_model(poison_x,poison_edge_index,poison_edge_weights)
train_attach_rate = (output.argmax(dim=1)[idx_attach]==args.target_class).float().mean()
print("target class rate on Vs: {:.4f}".format(train_attach_rate)) 

train_attach_rate = (poison_labels[idx_attach]==args.target_class).float().mean()
print("target class rate on Vs: {:.4f}".format(train_attach_rate)) 
#%%
from torch_sparse import SparseTensor
from baseline_atk import baseline_attack_parser, attack_baseline_construct
induct_edge_index = torch.cat([poison_edge_index,mask_edge_index],dim=1)
induct_edge_weights = torch.cat([poison_edge_weights,torch.ones([mask_edge_index.shape[1]],dtype=torch.float,device=device)])
clean_acc = test_model.test(poison_x,induct_edge_index,induct_edge_weights,data.y,idx_clean_test)
# test_model = test_model.cpu()

print("accuracy on clean test nodes: {:.4f}".format(clean_acc))

# poison_x, poison_edge_index, poison_edge_weights, poison_labels = poison_x.to(device2), poison_edge_index.to(device2), poison_edge_weights.to(device2), poison_labels.to(device2)
# model.trojan = model.trojan.cpu()
if(args.evaluate_mode == '1by1'):
    from torch_geometric.utils  import k_hop_subgraph
    overall_induct_edge_index, overall_induct_edge_weights = induct_edge_index.clone(),induct_edge_weights.clone()
    asr = 0
    for i, idx in enumerate(idx_atk):
        idx=int(idx)
        sub_induct_nodeset, sub_induct_edge_index, sub_mapping, sub_edge_mask  = k_hop_subgraph(node_idx = [idx], num_hops = 2, edge_index = overall_induct_edge_index, relabel_nodes=True) # sub_mapping means the index of [idx] in sub)nodeset
        ori_node_idx = sub_induct_nodeset[sub_mapping]
        relabeled_node_idx = sub_mapping
        sub_induct_edge_weights = torch.ones([sub_induct_edge_index.shape[1]]).to(device)
        # inject trigger on attack test nodes (idx_atk)'''
        if(args.attack_method == 'Basic'):
            induct_x, induct_edge_index,induct_edge_weights = model.inject_trigger(relabeled_node_idx,poison_x[sub_induct_nodeset],sub_induct_edge_index,sub_induct_edge_weights,device)
        elif(args.attack_method == 'Rand_Gene' or args.attack_method == 'Rand_Samp'):
            induct_x, induct_edge_index,induct_edge_weights = model.inject_trigger_rand(relabeled_node_idx,poison_x[sub_induct_nodeset],sub_induct_edge_index,sub_induct_edge_weights,data.y)
        elif(args.attack_method == 'None'):
            induct_x, induct_edge_index,induct_edge_weights = poison_x[sub_induct_nodeset],sub_induct_edge_index,sub_induct_edge_weights
        elif(args.attack_method == 'TDGIA'):
            induct_x, induct_edge_index,induct_edge_weights = poison_x[sub_induct_nodeset],sub_induct_edge_index,sub_induct_edge_weights
            atk_param = baseline_attack_parser(args,device)
            atk_param['n_inject_max'] = 3
            atk_param['n_edge_max'] = 1
            attacker = attack_baseline_construct(args,atk_param)
            sp_induct_adj = SparseTensor(row=induct_edge_index[0], col=induct_edge_index[1], value=induct_edge_weights)

            # induct_adj = to_dense_adj(induct_edge_index,edge_attr=induct_edge_weights)[0]
            # print(induct_adj.to_sparse())
            # sp_induct_adj = induct_adj.to_sparse()
            # print(sp_induct_adj.has_value())
            induct_adj, x_attack = attacker.attack(model=surrogate_model,
                                                    adj=sp_induct_adj,
                                                    features=induct_x,
                                                    target_idx=torch.tensor([relabeled_node_idx]),
                                                    labels=None)
            induct_x = torch.cat([induct_x,x_attack],dim=0)
            induct_edge_index,induct_edge_weights = dense_to_sparse(induct_adj.to_dense())
        induct_x, induct_edge_index,induct_edge_weights = induct_x.clone().detach(), induct_edge_index.clone().detach(),induct_edge_weights.clone().detach()
        # # do pruning in test datas'''
        if(args.defense_mode == 'prune' or args.defense_mode == 'isolate'):
            induct_edge_index,induct_edge_weights = prune_unrelated_edge(args,induct_edge_index,induct_edge_weights,induct_x,device)
        # attack evaluation
        
        # test_model = test_model.to(device)
        output = test_model(induct_x,induct_edge_index,induct_edge_weights)
        train_attach_rate = (output.argmax(dim=1)[relabeled_node_idx]==args.target_class).float().mean()
        print("Node {}: {}".format(i, idx))
        print("ASR: {:.4f}".format(train_attach_rate))
        asr += train_attach_rate
        # ca = test_model.test(induct_x,induct_edge_index,induct_edge_weights,data.y,idx_clean_test)
        # print("CA: {:.4f}".format(ca))
    asr = asr/(idx_atk.shape[0])
    print("Overall ASR: {:.4f}".format(asr))
elif(args.evaluate_mode == 'overall'):
    # %% inject trigger on attack test nodes (idx_atk)'''
    if(args.attack_method == 'Basic'):
        induct_x, induct_edge_index,induct_edge_weights = model.inject_trigger(idx_atk,poison_x,induct_edge_index,induct_edge_weights,device)
    elif(args.attack_method == 'Rand_Gene' or args.attack_method == 'Rand_Samp'):
        induct_x, induct_edge_index,induct_edge_weights = model.inject_trigger_rand(idx_atk,poison_x,induct_edge_index,induct_edge_weights,data.y)
    elif(args.attack_method == 'None'):
        induct_x, induct_edge_index,induct_edge_weights = poison_x,induct_edge_index,induct_edge_weights
    elif(args.attack_method == 'TDGIA' or args.attack_method == 'AGIA'):
        induct_x, induct_edge_index,induct_edge_weights = poison_x,data.edge_index,induct_edge_weights
        induct_edge_weights = torch.ones([data.edge_index.shape[1]],device=device,dtype=torch.float)
        atk_param = baseline_attack_parser(args,device)
        atk_param['n_inject_max'] = idx_atk.shape[0] * 3
        atk_param['n_edge_max'] = 1
        attacker = attack_baseline_construct(args,atk_param)
        sp_induct_adj = SparseTensor(row=induct_edge_index[0], col=induct_edge_index[1], value=induct_edge_weights)
        print(induct_edge_weights.shape)
        print(induct_x.shape)
        # induct_adj = to_dense_adj(induct_edge_index,edge_attr=induct_edge_weights)[0]
        # print(induct_adj.to_sparse())
        # sp_induct_adj = induct_adj.to_sparse()
        # print(sp_induct_adj.has_value())
        induct_adj, x_attack = attacker.attack(model=surrogate_model,
                                                adj=sp_induct_adj,
                                                features=induct_x,
                                                target_idx=idx_atk,
                                                labels=None)
        induct_x = torch.cat([induct_x,x_attack],dim=0)
        induct_edge_index,induct_edge_weights = dense_to_sparse(induct_adj.to_dense())
    induct_x, induct_edge_index,induct_edge_weights = induct_x.clone().detach(), induct_edge_index.clone().detach(),induct_edge_weights.clone().detach()
    # do pruning in test datas'''
    if(args.defense_mode == 'prune' or args.defense_mode == 'isolate'):
        induct_edge_index,induct_edge_weights = prune_unrelated_edge(args,induct_edge_index,induct_edge_weights,induct_x,device)
    # attack evaluation

    # test_model = test_model.to(device)
    output = test_model(induct_x,induct_edge_index,induct_edge_weights)
    if(args.attack_method == 'TDGIA' or args.attack_method == 'AGIA'):
        train_attach_rate = (output.argmax(dim=1)[idx_atk]!=data.y[idx_atk]).float().mean()
    else:
        train_attach_rate = (output.argmax(dim=1)[idx_atk]==args.target_class).float().mean()
    print("ASR: {:.4f}".format(train_attach_rate))
    ca = test_model.test(induct_x,induct_edge_index,induct_edge_weights,data.y,idx_clean_test)
    print("CA: {:.4f}".format(ca))

Namespace(attack_method='Basic', cuda=True, dataset='Cora', debug=True, defense_mode='guard', device_id=0, dis_weight=1, dropout=0.5, epochs=200, evaluate_mode='overall', hidden=32, homo_boost_thrd=0.5, homo_loss_weight=3, inner=1, lr=0.01, model='GCN', no_cuda=False, prune_thr=0.15, seed=10, selection_method='none', target_class=0, target_loss_weight=1, test_model='GCN', thrd=0.5, train_lr=0.01, trigger_prob=0.5, trigger_size=3, trojan_epochs=200, vs_ratio=0.005, weight_decay=0.0005)
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Length of training set: 541
Training benign model Finished!
Total time elapsed: 1.8465s
Benign CA: 0.8370
Epoch 0, loss_inner: 1.94621, loss_target: 1.93708, homo loss: 0.50406 
acc_train_clean: 0.1460, ASR_train_attach: 0.0000, ASR_train_outter: 0.0019
Epoch 10, loss_inner: 1.75595, loss_target: 1.00523, homo loss: 0.36427 
acc_train_clean: 0.2828, ASR_train_attach: 1.0000, ASR_train_outter: 0.9981

In [2]:
import torch.nn.functional as F
from torch_geometric.utils import to_dense_adj
poison_adj_dense = to_dense_adj(poison_edge_index)
def edge_sim_analysis(edge_index, features):
    sims = []
    for (u,v) in edge_index:
        sims.append(float(F.cosine_similarity(features[u].unsqueeze(0),features[v].unsqueeze(0))))
    sims = np.array(sims)
    # print(f"mean: {sims.mean()}, <0.1: {sum(sims<0.1)}/{sims.shape[0]}")
    return sims

bkd_nids = list(range(data.x.shape[0],poison_x.shape[0]))
for nid in idx_attach:
    # polished_dr_test = copy.deepcopy(bkd_dr_test)
    # polished_adj_nodes = polished_dr_test.data['mat_adj'].to_dense()[nid].nonzero()
    polished_adj_nodes = poison_adj_dense[0][nid].nonzero()
    # bkd_nids = list(range(poison_x.shape[0],induct_x.shape[0]))
    for v in polished_adj_nodes:
        v = int(v)
        if(v in bkd_nids):
            u = nid
            print(nid,v)
            print(F.cosine_similarity(poison_x[u].unsqueeze(0),poison_x[v].unsqueeze(0)))

tensor(1032, device='cuda:0') 2708
tensor([0.4862], device='cuda:0')
tensor(1201, device='cuda:0') 2711
tensor([0.5345], device='cuda:0')
tensor(760, device='cuda:0') 2714
tensor([0.5334], device='cuda:0')
tensor(2567, device='cuda:0') 2717
tensor([0.5273], device='cuda:0')
tensor(1147, device='cuda:0') 2720
tensor([0.5202], device='cuda:0')
tensor(2063, device='cuda:0') 2723
tensor([0.5417], device='cuda:0')
tensor(2574, device='cuda:0') 2726
tensor([0.5074], device='cuda:0')
tensor(1535, device='cuda:0') 2729
tensor([0.5272], device='cuda:0')
tensor(843, device='cuda:0') 2732
tensor([0.5263], device='cuda:0')
tensor(801, device='cuda:0') 2735
tensor([0.5114], device='cuda:0')


In [4]:
import torch.nn.functional as F
from torch_geometric.utils import to_dense_adj
induct_adj_dense = to_dense_adj(induct_edge_index)
def edge_sim_analysis(edge_index, features):
    sims = []
    for (u,v) in edge_index:
        sims.append(float(F.cosine_similarity(features[u].unsqueeze(0),features[v].unsqueeze(0))))
    sims = np.array(sims)
    # print(f"mean: {sims.mean()}, <0.1: {sum(sims<0.1)}/{sims.shape[0]}")
    return sims

bkd_nids = induct_x.shape[0] - poison_x.shape[0]
for nid in idx_atk:
    # polished_dr_test = copy.deepcopy(bkd_dr_test)
    # polished_adj_nodes = polished_dr_test.data['mat_adj'].to_dense()[nid].nonzero()
    polished_adj_nodes = induct_adj_dense[0][nid].nonzero()
    bkd_nids = list(range(poison_x.shape[0],induct_x.shape[0]))
    for v in polished_adj_nodes:
        v = int(v)
        if(v in bkd_nids):
            u = nid
            print(nid,v)
            print(F.cosine_similarity(induct_x[u].unsqueeze(0),induct_x[v].unsqueeze(0)))

tensor(1370, device='cuda:0') 2738
tensor([0.4264], device='cuda:0')
tensor(1376, device='cuda:0') 2741
tensor([0.4098], device='cuda:0')
tensor(1378, device='cuda:0') 2744
tensor([0.2614], device='cuda:0')
tensor(1392, device='cuda:0') 2747
tensor([0.3986], device='cuda:0')
tensor(1393, device='cuda:0') 2750
tensor([0.1695], device='cuda:0')
tensor(1400, device='cuda:0') 2753
tensor([0.4181], device='cuda:0')
tensor(1416, device='cuda:0') 2756
tensor([0.3980], device='cuda:0')
tensor(1422, device='cuda:0') 2759
tensor([0.3949], device='cuda:0')
tensor(1423, device='cuda:0') 2762
tensor([0.3337], device='cuda:0')
tensor(1425, device='cuda:0') 2765
tensor([0.3992], device='cuda:0')
tensor(1429, device='cuda:0') 2768
tensor([0.3847], device='cuda:0')
tensor(1434, device='cuda:0') 2771
tensor([0.3863], device='cuda:0')
tensor(1436, device='cuda:0') 2774
tensor([0.3059], device='cuda:0')
tensor(1437, device='cuda:0') 2777
tensor([0.3060], device='cuda:0')
tensor(1441, device='cuda:0') 2780